In [353]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipdb

In [354]:
pd.set_option('display.max_columns', None)
#pd.reset_option('display.max_columns')

In [355]:
#load in all my season data
season09=pd.read_csv('data/season-0910.csv')
season10=pd.read_csv('data/season-1011.csv')
season11=pd.read_csv('data/season-1112.csv')
season12=pd.read_csv('data/season-1213.csv')
season13=pd.read_csv('data/season-1314.csv')
season14=pd.read_csv('data/season-1415.csv')
season15=pd.read_csv('data/season-1516.csv')
season16=pd.read_csv('data/season-1617.csv')
season17=pd.read_csv('data/season-1718.csv')

In [356]:
seasons=[season09,season10,season11,season12,season13,season14,season15,season16,season17]

I will prepare features by each table separately. Teams change each year and I want to have data grouped by team. I will also want to have about 5 matches of data for each team to get some running aggregations

In [357]:
for i, table in enumerate(seasons):
    dates=table['Date'].apply(lambda x: len(x)).unique()
    if len(dates)==1:
        if dates[0]==8:
            table['Date']=pd.to_datetime(table['Date'], format='%d/%m/%y')
            
        elif dates[0]==10:
            table['Date']=pd.to_datetime(table['Date'], format='%d/%m/%Y')
        else:
            print(f"unexpected date format, check table {i}")
    else:
         print(f"more than 1 date format, check table {i}")

In [358]:
for i, season in enumerate(seasons):
    if season09['Date'].is_monotonic_increasing:
        print(f"Season number {i} is sorted by date")
    else:
        print(f"Season number {i} is not sorted by date")

Season number 0 is sorted by date
Season number 1 is sorted by date
Season number 2 is sorted by date
Season number 3 is sorted by date
Season number 4 is sorted by date
Season number 5 is sorted by date
Season number 6 is sorted by date
Season number 7 is sorted by date
Season number 8 is sorted by date


In [359]:
season09

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,2009-08-15,Aston Villa,Wigan,0,2,A,0,1,A,M Clattenburg,11,14,5,7,15,14,4,6,2,2,0,0
1,2009-08-15,Blackburn,Man City,0,2,A,0,1,A,M Dean,17,8,9,5,12,9,5,4,2,1,0,0
2,2009-08-15,Bolton,Sunderland,0,1,A,0,1,A,A Marriner,11,20,3,13,16,10,4,7,2,1,0,0
3,2009-08-15,Chelsea,Hull,2,1,H,1,1,D,A Wiley,26,7,12,3,13,15,12,4,1,2,0,0
4,2009-08-15,Everton,Arsenal,1,6,A,0,3,A,M Halsey,8,15,5,9,11,13,4,9,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2010-05-09,Everton,Portsmouth,1,0,H,0,0,D,P Walton,21,10,13,4,12,11,14,1,0,0,0,0
376,2010-05-09,Hull,Liverpool,0,0,D,0,0,D,A Marriner,11,19,6,7,15,11,2,6,1,0,0,0
377,2010-05-09,Man United,Stoke,4,0,H,2,0,H,M Clattenburg,18,4,13,4,10,4,10,2,2,0,0,0
378,2010-05-09,West Ham,Man City,1,1,D,1,1,D,H Webb,12,17,8,6,12,7,4,3,2,0,0,0


I need to restructure the data so I can get some running totals

In [360]:
#one-hot encode the result field
ftr_encoded = pd.get_dummies(season09['FTR'], prefix='', prefix_sep='')

In [361]:
# Adding the new one-hot encoded columns to the dataframe
season09 = pd.concat([season09, ftr_encoded], axis=1)

In [362]:
season09.rename(columns={'H': 'HomeWin', 'D': 'Draw', 'A': 'AwayWin'}, inplace=True)

In [363]:
season09.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'AwayWin', 'Draw', 'HomeWin'],
      dtype='object')

In [364]:
#Keeping most data except for away team, referee, half time result,  and away cards recieved by the opposite team.
#I dont think ill be ever using these running totals for the home team

homeMatches=season09[['Date', 'HomeTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'HR', 'HomeWin', 'Draw', 'AwayWin']]

In [365]:
homeMatches=homeMatches.copy()

homeMatches.rename(columns={'HomeTeam':'team',
                            'HomeWin':'win',
                            'Draw':'draw',
                            'AwayWin':'loss',
                            'FTHG':'goals',
                            'FTAG':'conceded',
                            'FTR':'result',
                            'HTHG':'half_goals',
                            'HTAG':'half_conceded',
                            'HS':'shots',
                            'AS':'shots_against',
                            'HST':'shots_target',
                            'AST':'shots_against_target',
                            'HF':'fouls',
                            'AF':'fouls_other_team',
                            'HC':'corners',
                            'AC':'corners_conceded',
                            'HY':'yellows',
                            'HR':'reds'}, inplace=True)
                            


In [366]:
awayMatches=season09[['Date', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'AY', 'AR', 'HomeWin', 'Draw', 'AwayWin']]

In [367]:
awayMatches=awayMatches.copy()
#same as above but with all the away stats swapped with the home stats
awayMatches.rename(columns={'AwayTeam':'team',
                            'AwayWin':'win',
                            'Draw':'draw',
                            'HomeWin':'loss',
                            'FTAG':'goals',
                            'FTHG':'conceded',
                            'FTR':'result',
                            'HTAG':'half_goals',
                            'HTHG':'half_conceded',
                            'AS':'shots',
                            'HS':'shots_against',
                            'AST':'shots_target',
                            'HST':'shots_against_target',
                            'AF':'fouls',
                            'HF':'fouls_other_team',
                            'AC':'corners',
                            'HC':'corners_conceded',
                            'AY':'yellows',
                            'AR':'reds'}, inplace=True)

In [368]:
awayMatches.reset_index(drop=True)

,Date,team,conceded,goals,result,half_conceded,half_goals,shots_against,shots,shots_against_target,shots_target,fouls_other_team,fouls,corners_conceded,corners,yellows,reds,loss,draw,win
0,2009-08-15,Wigan,0,2,A,0,1,11,14,5,7,15,14,4,6,2,0,False,False,True
1,2009-08-15,Man City,0,2,A,0,1,17,8,9,5,12,9,5,4,1,0,False,False,True
2,2009-08-15,Sunderland,0,1,A,0,1,11,20,3,13,16,10,4,7,1,0,False,False,True
3,2009-08-15,Hull,2,1,H,1,1,26,7,12,3,13,15,12,4,2,0,True,False,False
4,2009-08-15,Arsenal,1,6,A,0,3,8,15,5,9,11,13,4,9,0,0,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2010-05-09,Portsmouth,1,0,H,0,0,21,10,13,4,12,11,14,1,0,0,True,False,False
376,2010-05-09,Liverpool,0,0,D,0,0,11,19,6,7,15,11,2,6,0,0,False,True,False
377,2010-05-09,Stoke,4,0,H,2,0,18,4,13,4,10,4,10,2,0,0,True,False,False
378,2010-05-09,Man City,1,1,D,1,1,12,17,8,6,12,7,4,3,0,0,False,True,False


In [369]:
allMatches=pd.concat([homeMatches,awayMatches], axis=0).sort_values(by=['Date','team']).reset_index(drop=True)

In a previous project I got the running total of the first 5 games and then I added statistics for the 5 previous games and statistics for all previous games within a season.

I will also keep a count of how many matches a team has played so that I can filter using that stat.

In [370]:
allMatches['matchNumber']=allMatches.groupby('team')['Date'].transform('cumcount') +1

I want to get the days elapsed between the current match and the previous one

In [371]:
allMatches['daysSince']=allMatches.groupby('team')['Date'].transform(
    lambda x: x-x.shift(1)
)
    

In [372]:
# Define the columns to calculate running totals for
stats = ['win', 'draw', 'loss',
        'goals', 'conceded', 'half_goals', 'half_conceded',
        'shots', 'shots_against', 'shots_target', 'shots_against_target',
        'corners', 'corners_conceded',
        'fouls', 'fouls_other_team', 'yellows', 'reds']

# Loop through the stats and calculate the running totals
for stat in stats:
    allMatches[f'total.{stat}'] = allMatches.groupby('team')[stat].transform(
        lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
    )


In [373]:
# Define the columns to calculate running totals for
stats = ['win', 'draw', 'loss',
        'goals', 'conceded', 'half_goals', 'half_conceded',
        'shots', 'shots_against', 'shots_target', 'shots_against_target',
        'corners', 'corners_conceded',
        'fouls', 'fouls_other_team', 'yellows', 'reds']

# Loop through the stats and calculate the running totals
for stat in stats:
    allMatches[f'last3.{stat}'] = allMatches.groupby('team')[stat].transform(
        lambda x: x.shift(1).rolling(window=3, min_periods=1).mean()
    )


In [374]:
#sanity check
allMatches[allMatches['team']=='Arsenal'][['win', 'draw', 'loss',
                                           'total.win', 'total.draw', 'total.loss',
                                           'last3.win', 'last3.draw', 'last3.loss']].head(10)

,win,draw,loss,total.win,total.draw,total.loss,last3.win,last3.draw,last3.loss
0,True,False,False,<NA>,<NA>,<NA>,NaN,NaN,NaN
32,True,False,False,1.0,0.0,0.0,1.000000,0.000000,0.000000
52,False,False,True,1.0,0.0,0.0,1.000000,0.000000,0.000000
72,False,False,True,0.666667,0.0,0.333333,0.666667,0.000000,0.333333
92,True,False,False,0.5,0.0,0.5,0.333333,0.000000,0.666667
112,True,False,False,0.6,0.0,0.4,0.333333,0.000000,0.666667
142,True,False,False,0.666667,0.0,0.333333,0.666667,0.000000,0.333333
152,True,False,False,0.714286,0.0,0.285714,1.000000,0.000000,0.000000
184,False,True,False,0.75,0.0,0.25,1.000000,0.000000,0.000000
192,True,False,False,0.666667,0.111111,0.222222,0.666667,0.333333,0.000000


In [375]:
allMatches.columns

Index(['Date', 'team', 'goals', 'conceded', 'result', 'half_goals',
       'half_conceded', 'shots', 'shots_against', 'shots_target',
       'shots_against_target', 'fouls', 'fouls_other_team', 'corners',
       'corners_conceded', 'yellows', 'reds', 'win', 'draw', 'loss',
       'matchNumber', 'daysSince', 'total.win', 'total.draw', 'total.loss',
       'total.goals', 'total.conceded', 'total.half_goals',
       'total.half_conceded', 'total.shots', 'total.shots_against',
       'total.shots_target', 'total.shots_against_target', 'total.corners',
       'total.corners_conceded', 'total.fouls', 'total.fouls_other_team',
       'total.yellows', 'total.reds', 'last3.win', 'last3.draw', 'last3.loss',
       'last3.goals', 'last3.conceded', 'last3.half_goals',
       'last3.half_conceded', 'last3.shots', 'last3.shots_against',
       'last3.shots_target', 'last3.shots_against_target', 'last3.corners',
       'last3.corners_conceded', 'last3.fouls', 'last3.fouls_other_team',
       'last3.y

In [376]:
allMatches.drop(columns=['goals', 'conceded', 'result', 'half_goals',
       'half_conceded', 'shots', 'shots_against', 'shots_target',
       'shots_against_target', 'fouls', 'fouls_other_team', 'corners',
       'corners_conceded', 'yellows', 'reds', 'win', 'draw', 'loss'],inplace=True)

In [377]:
season09.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC',
       'HY', 'AY', 'HR', 'AR', 'AwayWin', 'Draw', 'HomeWin'],
      dtype='object')

In [378]:
season09=season09[['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'AwayWin', 'Draw', 'HomeWin']]

In [379]:
season09=season09.merge(
     allMatches.add_prefix('home.'), 
    how='left', left_on=['Date','HomeTeam'], right_on=['home.Date','home.team'])

season09.drop(columns=['home.Date','home.team'],inplace=True)

In [380]:
season09=season09.merge(
     allMatches.add_prefix('away.'), 
    how='left', left_on=['Date','AwayTeam'], right_on=['away.Date','away.team'])

season09.drop(columns=['away.Date','away.team'],inplace=True)

In [381]:
season09[(season09['HomeTeam']=='Arsenal') | (season09['AwayTeam']=='Arsenal')].columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'AwayWin', 'Draw', 'HomeWin',
       'home.matchNumber', 'home.daysSince', 'home.total.win',
       'home.total.draw', 'home.total.loss', 'home.total.goals',
       'home.total.conceded', 'home.total.half_goals',
       'home.total.half_conceded', 'home.total.shots',
       'home.total.shots_against', 'home.total.shots_target',
       'home.total.shots_against_target', 'home.total.corners',
       'home.total.corners_conceded', 'home.total.fouls',
       'home.total.fouls_other_team', 'home.total.yellows', 'home.total.reds',
       'home.last3.win', 'home.last3.draw', 'home.last3.loss',
       'home.last3.goals', 'home.last3.conceded', 'home.last3.half_goals',
       'home.last3.half_conceded', 'home.last3.shots',
       'home.last3.shots_against', 'home.last3.shots_target',
       'home.last3.shots_against_target', 'home.last3.corners',
       'home.last3.corners_conceded', 'home.last3.fouls',
       'home.last3.fouls_other_team', 'home.la

In [382]:
# sanity check

season09[(season09['HomeTeam']=='Arsenal') | (season09['AwayTeam']=='Arsenal')][['Date', 'HomeTeam', 'AwayTeam', 'FTR', 
                                                                                 'home.total.win', 'home.total.draw', 'home.total.loss', 'away.total.win', 'away.total.draw', 'away.total.loss',
                                                                                'home.last3.win', 'home.last3.draw', 'home.last3.loss', 'away.last3.win', 'away.last3.draw', 'away.last3.loss']].tail()

,Date,HomeTeam,AwayTeam,FTR,home.total.win,home.total.draw,home.total.loss,away.total.win,away.total.draw,away.total.loss,home.last3.win,home.last3.draw,home.last3.loss,away.last3.win,away.last3.draw,away.last3.loss
335,2010-04-14,Tottenham,Arsenal,H,0.53125,0.21875,0.25,0.666667,0.151515,0.181818,0.666667,0.000000,0.333333,0.666667,0.333333,0.000000
345,2010-04-18,Wigan,Arsenal,H,0.235294,0.235294,0.529412,0.647059,0.147059,0.205882,0.000000,0.333333,0.666667,0.333333,0.333333,0.333333
348,2010-04-24,Arsenal,Man City,D,0.628571,0.142857,0.228571,0.5,0.323529,0.176471,0.333333,0.000000,0.666667,0.666667,0.000000,0.333333
366,2010-05-03,Blackburn,Arsenal,H,0.305556,0.305556,0.388889,0.611111,0.166667,0.222222,0.000000,0.666667,0.333333,0.000000,0.333333,0.666667
370,2010-05-09,Arsenal,Fulham,H,0.594595,0.162162,0.243243,0.324324,0.27027,0.405405,0.000000,0.333333,0.666667,0.333333,0.000000,0.666667


In [383]:
#home v away performance by team (when home or away respectively)

season09['home.total.win.whenHome'] = season09.groupby('HomeTeam')['HomeWin'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)

season09['home.total.draw.whenHome'] = season09.groupby('HomeTeam')['Draw'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)

season09['home.total.loss.whenHome'] = season09.groupby('HomeTeam')['AwayWin'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)


season09['away.total.win.whenAway'] = season09.groupby('AwayTeam')['AwayWin'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)

season09['away.total.draw.whenAway'] = season09.groupby('AwayTeam')['Draw'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)

season09['away.total.loss.whenAway'] = season09.groupby('AwayTeam')['HomeWin'].transform(
    lambda x: x.shift(1).cumsum() / x.shift(1).expanding().count().replace(0, pd.NA)
)

In [384]:
season09.columns

Index(['Date', 'HomeTeam', 'AwayTeam', 'FTR', 'AwayWin', 'Draw', 'HomeWin',
       'home.matchNumber', 'home.daysSince', 'home.total.win',
       'home.total.draw', 'home.total.loss', 'home.total.goals',
       'home.total.conceded', 'home.total.half_goals',
       'home.total.half_conceded', 'home.total.shots',
       'home.total.shots_against', 'home.total.shots_target',
       'home.total.shots_against_target', 'home.total.corners',
       'home.total.corners_conceded', 'home.total.fouls',
       'home.total.fouls_other_team', 'home.total.yellows', 'home.total.reds',
       'home.last3.win', 'home.last3.draw', 'home.last3.loss',
       'home.last3.goals', 'home.last3.conceded', 'home.last3.half_goals',
       'home.last3.half_conceded', 'home.last3.shots',
       'home.last3.shots_against', 'home.last3.shots_target',
       'home.last3.shots_against_target', 'home.last3.corners',
       'home.last3.corners_conceded', 'home.last3.fouls',
       'home.last3.fouls_other_team', 'home.la

In [385]:
# sanity check

season09[(season09['HomeTeam']=='Arsenal') | (season09['AwayTeam']=='Arsenal')][['Date', 'HomeTeam', 'AwayTeam', 'FTR','home.total.win.whenHome', 'home.total.draw.whenHome',
       'home.total.loss.whenHome', 'away.total.win.whenAway',
       'away.total.draw.whenAway', 'away.total.loss.whenAway']].tail()

,Date,HomeTeam,AwayTeam,FTR,home.total.win.whenHome,home.total.draw.whenHome,home.total.loss.whenHome,away.total.win.whenAway,away.total.draw.whenAway,away.total.loss.whenAway
335,2010-04-14,Tottenham,Arsenal,H,0.6875,0.125,0.1875,0.5,0.25,0.25
345,2010-04-18,Wigan,Arsenal,H,0.294118,0.352941,0.352941,0.470588,0.235294,0.294118
348,2010-04-24,Arsenal,Man City,D,0.823529,0.058824,0.117647,0.352941,0.411765,0.235294
366,2010-05-03,Blackburn,Arsenal,H,0.5,0.333333,0.166667,0.444444,0.222222,0.333333
370,2010-05-09,Arsenal,Fulham,H,0.777778,0.111111,0.111111,0.055556,0.388889,0.555556


In [386]:
season09['season']='season09'

In [387]:
season09[season09['home.matchNumber']==4]

,Date,HomeTeam,AwayTeam,FTR,AwayWin,Draw,HomeWin,home.matchNumber,home.daysSince,home.total.win,home.total.draw,home.total.loss,home.total.goals,home.total.conceded,home.total.half_goals,home.total.half_conceded,home.total.shots,home.total.shots_against,home.total.shots_target,home.total.shots_against_target,home.total.corners,home.total.corners_conceded,home.total.fouls,home.total.fouls_other_team,home.total.yellows,home.total.reds,home.last3.win,home.last3.draw,home.last3.loss,home.last3.goals,home.last3.conceded,home.last3.half_goals,home.last3.half_conceded,home.last3.shots,home.last3.shots_against,home.last3.shots_target,home.last3.shots_against_target,home.last3.corners,home.last3.corners_conceded,home.last3.fouls,home.last3.fouls_other_team,home.last3.yellows,home.last3.reds,away.matchNumber,away.daysSince,away.total.win,away.total.draw,away.total.loss,away.total.goals,away.total.conceded,away.total.half_goals,away.total.half_conceded,away.total.shots,away.total.shots_against,away.total.shots_target,away.total.shots_against_target,away.total.corners,away.total.corners_conceded,away.total.fouls,away.total.fouls_other_team,away.total.yellows,away.total.reds,away.last3.win,away.last3.draw,away.last3.loss,away.last3.goals,away.last3.conceded,away.last3.half_goals,away.last3.half_conceded,away.last3.shots,away.last3.shots_against,away.last3.shots_target,away.last3.shots_against_target,away.last3.corners,away.last3.corners_conceded,away.last3.fouls,away.last3.fouls_other_team,away.last3.yellows,away.last3.reds,home.total.win.whenHome,home.total.draw.whenHome,home.total.loss.whenHome,away.total.win.whenAway,away.total.draw.whenAway,away.total.loss.whenAway,season
28,2009-08-29,Chelsea,Burnley,H,False,False,True,4,6 days,1.0,0.0,0.0,2.333333,0.666667,0.666667,0.666667,19.333333,5.0,8.333333,2.333333,11.0,3.0,11.0,13.333333,1.0,0.0,1.000000,0.000000,0.000000,2.333333,0.666667,0.666667,0.666667,19.333333,5.000000,8.333333,2.333333,11.000000,3.000000,11.000000,13.333333,1.000000,0.0,4,6 days,0.666667,0.0,0.333333,0.666667,0.666667,0.666667,0.666667,8.333333,15.666667,3.666667,8.333333,3.333333,7.333333,9.666667,13.333333,1.666667,0.0,0.666667,0.000000,0.333333,0.666667,0.666667,0.666667,0.666667,8.333333,15.666667,3.666667,8.333333,3.333333,7.333333,9.666667,13.333333,1.666667,0.0,1.0,0.0,0.0,0.0,0.0,1.0,season09
29,2009-08-29,Man United,Arsenal,H,False,False,True,4,7 days,0.666667,0.0,0.333333,2.0,0.333333,0.333333,0.333333,20.0,10.0,13.0,4.333333,10.0,2.0,11.0,8.666667,1.333333,0.0,0.666667,0.000000,0.333333,2.000000,0.333333,0.333333,0.333333,20.000000,10.000000,13.000000,4.333333,10.000000,2.000000,11.000000,8.666667,1.333333,0.0,3,7 days,1.0,0.0,0.0,5.0,1.0,2.5,0.5,17.0,8.5,12.5,4.5,8.5,4.0,11.0,10.5,0.5,0.0,1.000000,0.000000,0.000000,5.000000,1.000000,2.500000,0.500000,17.000000,8.500000,12.500000,4.500000,8.500000,4.000000,11.000000,10.500000,0.500000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,season09
30,2009-08-29,Stoke,Sunderland,H,False,False,True,4,7 days,0.333333,0.333333,0.333333,0.666667,1.333333,0.666667,0.666667,10.333333,11.666667,3.666667,8.0,3.666667,6.666667,12.333333,9.333333,2.0,0.0,0.333333,0.333333,0.333333,0.666667,1.333333,0.666667,0.666667,10.333333,11.666667,3.666667,8.000000,3.666667,6.666667,12.333333,9.333333,2.000000,0.0,4,7 days,0.666667,0.0,0.333333,1.333333,1.333333,1.0,0.333333,10.666667,16.333333,7.0,5.666667,2.666667,9.333333,12.0,13.333333,1.666667,0.0,0.666667,0.000000,0.333333,1.333333,1.333333,1.000000,0.333333,10.666667,16.333333,7.000000,5.666667,2.666667,9.333333,12.000000,13.333333,1.666667,0.0,1.0,0.0,0.0,1.0,0.0,0.0,season09
31,2009-08-29,Tottenham,Birmingham,H,False,False,True,4,6 days,1.0,0.0,0.0,3.0,1.0,1.333333,0.333333,17.333333,10.666667,11.0,6.0,6.333333,5.0,12.666667,17.333333,2.0,0.0,1.000000,0.000000,0.000000,3.000000,1.000000,1.333333,0.333333,17.333333,10.666667,11.000000,6.000000,6.333333,5.000000,12.666667,17.333333,2.000000,0.0,4,7 days,0.333333,0.333333,0.333333,0.333333,0.333333,0

In [390]:
season09[(season09['home.matchNumber']>=4) & (season09['away.matchNumber']>=4)]

,Date,HomeTeam,AwayTeam,FTR,AwayWin,Draw,HomeWin,home.matchNumber,home.daysSince,home.total.win,home.total.draw,home.total.loss,home.total.goals,home.total.conceded,home.total.half_goals,home.total.half_conceded,home.total.shots,home.total.shots_against,home.total.shots_target,home.total.shots_against_target,home.total.corners,home.total.corners_conceded,home.total.fouls,home.total.fouls_other_team,home.total.yellows,home.total.reds,home.last3.win,home.last3.draw,home.last3.loss,home.last3.goals,home.last3.conceded,home.last3.half_goals,home.last3.half_conceded,home.last3.shots,home.last3.shots_against,home.last3.shots_target,home.last3.shots_against_target,home.last3.corners,home.last3.corners_conceded,home.last3.fouls,home.last3.fouls_other_team,home.last3.yellows,home.last3.reds,away.matchNumber,away.daysSince,away.total.win,away.total.draw,away.total.loss,away.total.goals,away.total.conceded,away.total.half_goals,away.total.half_conceded,away.total.shots,away.total.shots_against,away.total.shots_target,away.total.shots_against_target,away.total.corners,away.total.corners_conceded,away.total.fouls,away.total.fouls_other_team,away.total.yellows,away.total.reds,away.last3.win,away.last3.draw,away.last3.loss,away.last3.goals,away.last3.conceded,away.last3.half_goals,away.last3.half_conceded,away.last3.shots,away.last3.shots_against,away.last3.shots_target,away.last3.shots_against_target,away.last3.corners,away.last3.corners_conceded,away.last3.fouls,away.last3.fouls_other_team,away.last3.yellows,away.last3.reds,home.total.win.whenHome,home.total.draw.whenHome,home.total.loss.whenHome,away.total.win.whenAway,away.total.draw.whenAway,away.total.loss.whenAway,season
28,2009-08-29,Chelsea,Burnley,H,False,False,True,4,6 days,1.0,0.0,0.0,2.333333,0.666667,0.666667,0.666667,19.333333,5.0,8.333333,2.333333,11.0,3.0,11.0,13.333333,1.0,0.0,1.000000,0.000000,0.000000,2.333333,0.666667,0.666667,0.666667,19.333333,5.000000,8.333333,2.333333,11.000000,3.000000,11.000000,13.333333,1.000000,0.000000,4,6 days,0.666667,0.0,0.333333,0.666667,0.666667,0.666667,0.666667,8.333333,15.666667,3.666667,8.333333,3.333333,7.333333,9.666667,13.333333,1.666667,0.0,0.666667,0.000000,0.333333,0.666667,0.666667,0.666667,0.666667,8.333333,15.666667,3.666667,8.333333,3.333333,7.333333,9.666667,13.333333,1.666667,0.000000,1.0,0.0,0.0,0.0,0.0,1.0,season09
30,2009-08-29,Stoke,Sunderland,H,False,False,True,4,7 days,0.333333,0.333333,0.333333,0.666667,1.333333,0.666667,0.666667,10.333333,11.666667,3.666667,8.0,3.666667,6.666667,12.333333,9.333333,2.0,0.0,0.333333,0.333333,0.333333,0.666667,1.333333,0.666667,0.666667,10.333333,11.666667,3.666667,8.000000,3.666667,6.666667,12.333333,9.333333,2.000000,0.000000,4,7 days,0.666667,0.0,0.333333,1.333333,1.333333,1.0,0.333333,10.666667,16.333333,7.0,5.666667,2.666667,9.333333,12.0,13.333333,1.666667,0.0,0.666667,0.000000,0.333333,1.333333,1.333333,1.000000,0.333333,10.666667,16.333333,7.000000,5.666667,2.666667,9.333333,12.000000,13.333333,1.666667,0.000000,1.0,0.0,0.0,1.0,0.0,0.0,season09
31,2009-08-29,Tottenham,Birmingham,H,False,False,True,4,6 days,1.0,0.0,0.0,3.0,1.0,1.333333,0.333333,17.333333,10.666667,11.0,6.0,6.333333,5.0,12.666667,17.333333,2.0,0.0,1.000000,0.000000,0.000000,3.000000,1.000000,1.333333,0.333333,17.333333,10.666667,11.000000,6.000000,6.333333,5.000000,12.666667,17.333333,2.000000,0.000000,4,7 days,0.333333,0.333333,0.333333,0.333333,0.333333,0.0,0.333333,7.666667,16.0,5.0,8.0,3.0,7.0,8.666667,15.333333,0.333333,0.0,0.333333,0.333333,0.333333,0.333333,0.333333,0.000000,0.333333,7.666667,16.000000,5.000000,8.000000,3.000000,7.000000,8.666667,15.333333,0.333333,0.000000,1.0,0.0,0.0,0.0,0.0,1.0,season09
32,2009-08-29,Wolves,Hull,D,False,True,False,4,7 days,0.333333,0.0,0.666667,0.333333,1.0,0.333333,0.666667,12.666667,16.666667,5.333333,9.666667,6.0,5.0,13.666667,6.666667,1.333333,0.0,0.333333,0.000000,0.666667,0.333333,1.000000,0.333333,0.666667,12.666667,16.666667,5.333333,9.666667,6.000000,5.000000,13

In [393]:
season09

,Date,HomeTeam,AwayTeam,FTR,AwayWin,Draw,HomeWin,home.matchNumber,home.daysSince,home.total.win,home.total.draw,home.total.loss,home.total.goals,home.total.conceded,home.total.half_goals,home.total.half_conceded,home.total.shots,home.total.shots_against,home.total.shots_target,home.total.shots_against_target,home.total.corners,home.total.corners_conceded,home.total.fouls,home.total.fouls_other_team,home.total.yellows,home.total.reds,home.last3.win,home.last3.draw,home.last3.loss,home.last3.goals,home.last3.conceded,home.last3.half_goals,home.last3.half_conceded,home.last3.shots,home.last3.shots_against,home.last3.shots_target,home.last3.shots_against_target,home.last3.corners,home.last3.corners_conceded,home.last3.fouls,home.last3.fouls_other_team,home.last3.yellows,home.last3.reds,away.matchNumber,away.daysSince,away.total.win,away.total.draw,away.total.loss,away.total.goals,away.total.conceded,away.total.half_goals,away.total.half_conceded,away.total.shots,away.total.shots_against,away.total.shots_target,away.total.shots_against_target,away.total.corners,away.total.corners_conceded,away.total.fouls,away.total.fouls_other_team,away.total.yellows,away.total.reds,away.last3.win,away.last3.draw,away.last3.loss,away.last3.goals,away.last3.conceded,away.last3.half_goals,away.last3.half_conceded,away.last3.shots,away.last3.shots_against,away.last3.shots_target,away.last3.shots_against_target,away.last3.corners,away.last3.corners_conceded,away.last3.fouls,away.last3.fouls_other_team,away.last3.yellows,away.last3.reds,home.total.win.whenHome,home.total.draw.whenHome,home.total.loss.whenHome,away.total.win.whenAway,away.total.draw.whenAway,away.total.loss.whenAway,season
0,2009-08-15,Aston Villa,Wigan,A,True,False,False,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,season09
1,2009-08-15,Blackburn,Man City,A,True,False,False,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,season09
2,2009-08-15,Bolton,Sunderland,A,True,False,False,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,season09
3,2009-08-15,Chelsea,Hull,H,False,False,True,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,season09
4,2009-08-15,Everton,Arsenal,A,True,False,False,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,season09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...